\begin{equation*}
\begin{array}{ccc}
\text{Carla Reyes} & \text{Jerónimo Deli} & \text{Jaime Uria} \\
\text{Frida Márquez} & \text{Andrés Schafler} & \text{Aranza Ibarra} \\
\text{Javier Alexis Lopez} & \text{Armando Limón} & \text{Gustavo Escobar} \\
\text{Victor Hugo Esparza} & \text{Manuel McCadden} & \text{Diego García-Gayou} \\
\text{Maria José Velázquez} & \text{Juan Pablo Cordero} & \text{Jorge Esteban Ramirez} \\
& \text{Javier Matrínez del Campo} &
\end{array}
\end{equation*}

# Objetivo
El objetivo del siguiente reporte es encontrar relaciones entre los diversos actores que aparecen en los *Pandora Papers*. La intención es mostrar redes de corrupción y fallas en las normas de transparencia y justicia fiscal. Para ello, llevaremos a cabo un análisis exploratorio de una base de datos creada por el ICIJ en Neo4j a partir de los documentos previamente mencionados.

# Introducción
Los *Pandora Papers* filtrados por el Consorcio Internacional de Periodistas de Investigación (ICIJ) representan una de las filtraciones más grandes y reveladoras de datos financieros en la historia, exponiendo un universo oculto de riqueza, evasión fiscal y secretos financieros a nivel global. Este reportaje busca desentrañar las complejas redes de compañías offshore y los esquemas utilizados por políticos, magnates y celebridades para ocultar sus activos. A través de un análisis exhaustivo de la base de datos creada por ICIJ, exploraremos las implicaciones éticas, legales y económicas de estas prácticas en diversas jurisdicciones.

# Desarrollo
## Instalación de la base de datos

El primer paso fue conectarnos a la base de datos y cargar toda su información.

In [1]:
from py2neo import Graph

# Conexión usando el nombre del servicio Docker
url = "bolt://neo4j:7687"  # Cambiado de localhost a neo4j
username = "neo4j"  # Usa el nombre de usuario correcto si la autenticación está habilitada
password = "Neo4jneo4j"  # Usa la contraseña correcta si la autenticación está habilitada
graph = Graph(url, auth=(username, password))

In [2]:
import_cmd = """
CALL apoc.load.jsonArray("https://gist.githubusercontent.com/jexp/8afb65325bf99014c8de68a9511a549b/raw/pandora.json") yield value
WITH collect(value) as values
UNWIND range(0, size(values)-1) as set
WITH apoc.convert.toMap(values[set]) as value, set
UNWIND value.nodes as n
CALL apoc.merge.node(n.data.categories, {node_id: n.data.properties.node_id},n.data.properties) YIELD node
SET node.id=set+"-"+n.id
RETURN count(*);
"""

result = graph.run(import_cmd)
print(result.data())

[{'count(*)': 343}]


In [3]:
import_cmd = """
CALL apoc.load.jsonArray("https://gist.githubusercontent.com/jexp/8afb65325bf99014c8de68a9511a549b/raw/pandora.json") yield value
WITH collect(value) as values
UNWIND range(0, size(values)-1) as set
WITH apoc.convert.toMap(values[set]) as value, set
UNWIND value.edges as e
MATCH (n) where n.id=set+"-"+e.source
MATCH (m) where m.id=set+"-"+e.target
CALL apoc.create.relationship(n, e.data.type, 
apoc.map.clean(e.data.properties,["edge_id","power_player_profile_id"],[]) ,m) yield rel
RETURN count(*);
"""

result = graph.run(import_cmd)
print(result.data())

[{'count(*)': 267}]


In [4]:
import_cmd = """
MATCH (e:Entity)
WITH e.name as name, collect(e) as nodes
WHERE size(nodes) > 1
CALL apoc.refactor.mergeNodes(nodes, {properties:"discard", mergeRels:true}) YIELD node return count(*);
"""

result = graph.run(import_cmd)
print(result.data())

[{'count(*)': 1}]


In [5]:
import_cmd = """
MATCH (n:Entity)
MERGE (c:Country {name: coalesce(n.jurisdiction,"UNKNOWN")})
MERGE (n)-[:RESIDES_IN]->(c);
"""

result = graph.run(import_cmd)
print(result.data())

[]


In [6]:
import_cmd = """
MATCH (o:Officer)
MERGE (c:Country {name: coalesce(o.country, "UNKNOWN")})
MERGE (o)-[:CITIZEN_OF]->(c);
"""

result = graph.run(import_cmd)
print(result.data())

[]


In [7]:
import_cmd = """
MATCH (n:Entity)
MERGE (p:Provider {name: coalesce(n.provider,"UNKNOWN")})
MERGE (n)-[:PROVIDED_BY]->(p);
"""

result = graph.run(import_cmd)
print(result.data())

[]


## Análisis

El primer paso de nuestro análisis fue identificar en qué país se concentra mayor parte de las empresas fantasma. Esto nos podría indicar fallas en la legislación fiscal de dicha nación.

In [8]:
import_cmd = """
MATCH (e:Entity)--(c2:Country)
WITH c2.name AS EntityCountry, COUNT(DISTINCT e) AS NumberOfEntities
RETURN EntityCountry, NumberOfEntities
ORDER BY NumberOfEntities DESC;
"""

result = graph.run(import_cmd)
result

EntityCountry,NumberOfEntities
British Virgin Islands,180
Panama,38
Seychelles,4


Ejecutando esta consulta, podemos observar que British Virgin Islands es el territorio con mayor número de empresas fantasmas. Cuenta con 180, 142 más que el país siguiente. Por esto, decidimos concentrarnos en explorar su legislación.

Las Islas Vírgenes Británicas son un destino popular para la creación de empresas offshore con el objetivo de ocultar activos y evadir impuestos. Sus ventajas incluyen beneficios fiscales, un alto nivel de confidencialidad, servicios financieros desarrollados, estabilidad política y financiera. Además, las empresas registradas en BVI que no realizan operaciones comerciales dentro del territorio están exentas de impuestos sobre ganancias de capital, impuesto sobre la renta, entre otros.

La ausencia de retenciones en BVI es el principal incentivo para que líderes mundiales y empresarios establezcan sus empresas allí. No hay impuestos sobre el pago de dividendos, lo que hace que las distribuciones de beneficios sean menos costosas que en otros países. Además, los nombres de los beneficiarios, directores y accionistas no aparecen en registros públicos debido a la legislación vigente.

Nos enfocamos exclusivamente en las empresas que operan bajo esta deficiente legislación fiscal, con la intención de encontrar relaciones entre las distintas figuras públicas. Buscamos determinar si existen empresas con más de un dueño reconocido.

In [9]:
import_cmd = """
MATCH (o:Officer)-->(e:Entity)-->(c:Country)
WITH e, c, COUNT(DISTINCT o) AS numOfficers
WHERE c.name = "British Virgin Islands" AND numOfficers > 1
RETURN e.name AS EntityName, c.name AS EntityCountry, numOfficers
ORDER BY numOfficers DESC;
"""
result = graph.run(import_cmd)
for record in result:
    print(record)

'Luntrel Investments Limited '	'British Virgin Islands'	4
'Milrun International Limited'	'British Virgin Islands'	2
'Birtan Ltd.'	'British Virgin Islands'	2
'Albatros Investing Ltd.'	'British Virgin Islands'	2
'Barlock Investments Limited'	'British Virgin Islands'	2
'Wesmar Finance Limited'	'British Virgin Islands'	2
'Milestars Invest Ltd.'	'British Virgin Islands'	2
'ROMANSTONE INTERNATIONAL LIMITED'	'British Virgin Islands'	2
'International Consultants in Economic and Social Research Inc'	'British Virgin Islands'	2


Al ejecutar esta consulta, notamos que existen nueve empresas con más de un involucrado. Tras descubrir esto, quisimos investigar vínculos internacionales. Específicamente, diseñamos una consulta para encontrar empresas vinculadas con más de una persona, y que los involucrados fueran de diferentes países. Explorando la base descartamos esta posibilidad. Por esto, no exploramos redes internacionales.

In [10]:
import_cmd = """
MATCH (c2:Country)<--(o:Officer)-->(e:Entity)-->(c:Country)
WHERE c.name = "British Virgin Islands"
WITH e, c, Count(DISTINCT c2.name) AS OfficerCountries
WHERE OfficerCountries > 1
RETURN e.name AS EntityName, c.name AS EntityCountry, OfficerCountries
ORDER BY OfficerCountries DESC;

"""

result = graph.run(import_cmd)
result

(No data)

Una vez que sabemos que para cada empresa, todos los asociados son del mismo país, queremos explorar las relaciones entre ellos. Buscamos las empresas con mayor cantidad de dueños, con la intención de encontrar redes de empresarios y políticos relacionados. Filtramos sólo las cinco empresas con más agentes vinculados, y descubrimos que tres de ellas estaban conectadas con figuras públicas panameñas.

In [11]:
import_cmd = """
MATCH (c2:Country)<--(o:Officer)-->(e:Entity)-->(c:Country)
WITH e, c,c2, COUNT(DISTINCT o) AS numOfficers
WHERE c.name = "British Virgin Islands" AND numOfficers > 1
RETURN e.name AS EntityName, c.name AS EntityCountry, c2.name, numOfficers
ORDER BY numOfficers DESC
LIMIT 5;
"""
result = graph.run(import_cmd)
result

EntityName,EntityCountry,c2.name,numOfficers
Luntrel Investments Limited,British Virgin Islands,Panama,4
Albatros Investing Ltd.,British Virgin Islands,Panama,2
Barlock Investments Limited,British Virgin Islands,Panama,2


Nos preguntamos si esto era casualidad, sin embargo, al hacer un análisis más general, Panamá resultó ser el país de origen con mayor número de empresarios relacionados a empresas fantasmas mundialmente.

In [12]:
import_cmd = """
MATCH (country:Country)
OPTIONAL MATCH (officer:Officer)-[c:CITIZEN_OF]->(country)
OPTIONAL MATCH (officer:Officer)-[r:RESIDES_IN]->(country)
WITH country, count(c) + count(r) AS total_officers
RETURN country.name AS country, total_officers
ORDER BY total_officers DESC;
"""
result = graph.run(import_cmd)
result

country,total_officers
Panama,9
Kenya,6
Sri Lanka,4


Retomando el análisis de empresas en las British Virgin Islands, nos concentraremos en los dueños panameños de las empresas previamente mencionadas.


In [13]:
import_cmd = """
MATCH (c1:Country)<--(o:Officer)-->(e:Entity)
WHERE e.name = "Luntrel Investments Limited "
RETURN o.name
"""
result = graph.run(import_cmd)
result

o.name
María Enriqueta Pérez Balladares de Iglesias
Dora María Pérez Balladares Boyd
Ernesto Pérez Balladares


In [14]:
import_cmd = """
MATCH (c1:Country)<--(o:Officer)-->(e:Entity)
WHERE e.name = "Albatros Investing Ltd."
RETURN o.name
"""
result = graph.run(import_cmd)
result

o.name
Ricardo Alberto Martinelli Linares
Luis Enrique Martinelli Linares
Luis Enrique Martinelli Linares


In [15]:
import_cmd = """
MATCH (c1:Country)<--(o:Officer)-->(e:Entity)
WHERE e.name = "Milestars Invest Ltd."
RETURN o.name
"""
result = graph.run(import_cmd)
result

o.name
Luis Enrique Martinelli Linares
Ricardo Alberto Martinelli Linares
Luis Enrique Martinelli Linares


Si bien en las 3 empresas  encontramos nombres que podríamos vincular en diversos asuntos de relevancia sociopolítica y económica, Luntrel Investment Limited destaca, pues se aprecia que los miembros que componen su estructura comparten apellido. Profundizaremos sobre este asunto. 

In [16]:
import_cmd = """
MATCH (o:Officer)-->(e:Entity)
WHERE toLower(o.name) CONTAINS 'balladares'
RETURN distinct(e.name);
"""
result = graph.run(import_cmd)
for record in result:
    print(record)

'Berkland Investments Ltd'
'Barlock Investments Limited'
'Luntrel Investments Limited '
'Bradeton Investments Limited'


En total, son 4 las empresas vinculadas con este apellido, dispersas por el mundo. Si bien esto es un comportamiento sospechoso y del cual podríamos inferir la existencia de movimientos ilícitos, conviene hacer una investigación más profunda tanto de la familia, así como de estas empresas, y a partir de esto hacer las asociaciones relevantes. 

### Familia Pérez Balladares

La familia Pérez Balladares es una de las más destacadas en Panamá, conocida por su influencia en el ámbito político y económico del país. Sin embargo, también ha estado involucrada en múltiples controversias, especialmente relacionadas con el uso de estructuras financieras offshore, como lo revelaron los Pandora Papers.

### Ernesto Pérez Balladares

Ernesto Pérez Balladares fue presidente de Panamá entre 1994 y 1999. Durante su presidencia, implementó una serie de reformas económicas, incluyendo la privatización de empresas estatales y la promoción de políticas de libre mercado. No obstante, su mandato estuvo marcado por acusaciones de corrupción y, posteriormente, enfrentó múltiples investigaciones. Según los Pandora Papers, utilizó empresas offshore para gestionar su riqueza y evitar impuestos. Estas entidades financieras, establecidas en paraísos fiscales, plantean serias dudas sobre la legalidad y ética de sus acciones financieras. Tanto durante como después de su mandato, estas estructuras fueron utilizadas para mantener y mover sus activos fuera del escrutinio público.

### Dora María Pérez Balladares Boyd

Dora María Pérez Balladares Boyd, hermana de Ernesto, también ha sido implicada en actividades financieras offshore. De acuerdo con los Pandora Papers, es beneficiaria de varias entidades registradas en las Islas Vírgenes Británicas (BVI), incluyendo Ashlock International Limited y Barlock Investments Limited. Estas entidades se utilizaron para ocultar activos financieros, aprovechando las ventajas fiscales y la confidencialidad que ofrecen estas jurisdicciones.

### Isabella Pérez Balladares de Pretelt

Isabella Pérez Balladares de Pretelt, hija de Ernesto Pérez Balladares, también está vinculada a actividades offshore. Es beneficiaria de Luntrel Investments Limited, una empresa registrada en las Islas Vírgenes Británicas. Su participación en estas estructuras refleja la continuidad de las prácticas de gestión de activos a través de paraísos fiscales entre las generaciones de la familia.

### María Enriqueta Pérez Balladares de Iglesias

María Enriqueta Pérez Balladares de Iglesias, madre de Ernesto Pérez Balladares, es otro miembro prominente de la familia. También ha sido mencionada en relación con varias entidades offshore. Aunque los detalles específicos sobre sus actividades son limitados, su inclusión en los registros de los Pandora Papers sugiere una participación significativa en la gestión de la riqueza familiar a través de estas estructuras.

### La relación entre familias

Las familias Martinelli Linares y Pérez Balladares son los protagonístas de este escándalo y tienen una relación significativa a través de los vínculos políticos y personales en Panamá. Ernesto Pérez Balladares fue presidente de Panamá de 1994 a 1999, y diez años después, Ricardo Martinelli llegó al mismo puesto, aunque representando al Partido Cambio Democrático (CD). Ambos personajes coincidieron antes de ser miembros del partido, ya que ambos trabajaron en la misma área de Citibank.

En una entrevista para TVN, Pérez Balladares declaró que el exmandatario Ricardo Martinelli se dedicó durante su gobierno a hacer negocios para él y su familia. Esta declaración muestra que, a pesar de ya no ser presidente, Ernesto seguía siendo beneficiado por el poder político de Panamá.

Estas relaciones ilustran la complejidad y la profundidad de la corrupción en la política panameña, y cómo influencias familiares pueden extenderse a lo largo de diferentes administraciones y sectors

# Conclusión 

El análisis de los Pandora Papers reveló que políticos, empresarios y celebridades utilizan compañías offshore para ocultar activos y evadir impuestos. Panamá destaca como un centro importante de empresas fantasma, lo que resalta la necesidad de investigar las conexiones políticas en este país. La familia Pérez Balladares es un ejemplo de cómo se usan estructuras financieras offshore para gestionar la riqueza familiar, cuestionando la ética y legalidad de estas acciones.

Las conexiones entre las familias Martinelli Linares y Pérez Balladares muestran cómo la influencia política y económica puede perpetuar la corrupción más allá de los mandatos presidenciales. Estos hallazgos subrayan la necesidad de reformas internacionales y una mayor supervisión fiscal para combatir estas prácticas y promover la transparencia.

Un aspecto clave en la investigación fue el uso de Neo4j, un manejador de bases de datos orientadas a grafos. Estas bases de datos permitieron analizar de manera eficiente las relaciones entre empresas, lugares y personas involucradas, facilitando la identificación de patrones complejos. La capacidad de las bases de datos de grafos para visualizar y analizar conexiones fue crucial para entender la magnitud y el alcance de las redes de evasión fiscal y corrupción. Este es un gran ejemplo de la aplicación de bases de datos de grafos en investigaciones complejas, demostrando su potencial para revelar estructuras ocultas en grandes conjuntos de datos.

Utilizando consultas de Neo4j hicimos un análisis exploratorio de la base. Comenzando con consultas generales, logramos obtener un panorama de la información. Así, encontramos patrones generales. Paulatinamente, profundizamos en las relaciones entre los distintos objetos, afinando nuestras consultas. Así, pudimos enfocar nuestra investigación, o que nos ayudó a descubrir insights valiosos.

El estudio resalta la complejidad de la evasión fiscal y los retos globales para erradicar estas actividades, destacando la importancia de una cooperación internacional más estrecha y medidas estrictas para asegurar la justicia fiscal. Los Pandora Papers exponen la magnitud de la evasión fiscal y la corrupción, y enfatizan la urgencia de reformar las normas fiscales y fortalecer la supervisión internacional para lograr un sistema financiero más justo y transparente.

# Bibliografía

* ICIJ. (2021). ``Ernesto Pérez Balladares''. Recuperado de \url{https://projects.icij.org/investigations/pandora-papers/power-players/es/player/ernesto-perez-balladares#:~:text=URL\%3A\%20https\%3A\%2F\%2Fprojects.icij.org\%2Finvestigations\%2Fpandora}

* ICIJ. (2018). ``Dora Ma. Boyd de Perez Balladares''. Recuperado de \url{https://offshoreleaks.icij.org/nodes/240053397}

* ICIJ. (2018). ``Luntrel Investments Limited''. Recuperado de \url{https://offshoreleaks.icij.org/nodes/240030531}

* CIDOB. ``Ricardo Martinelli Berrocal''. Recuperado de \url{https://www.cidob.org/biografias_lideres_politicos/america_central_y_caribe/panama/ricardo_martinelli_berrocal}

* CIDOB. (2016). ``Ernesto Pérez Balladares''. Recuperado de \url{https://www.cidob.org/ca/biografias_lideres_politicos/america_central_y_caribe/panama/ernesto_perez_balladares}


In [17]:
!jupyter nbconvert --to pdf "Pandora_Papers.ipynb"

[NbConvertApp] Converting notebook Pandora_Papers.ipynb to pdf
[NbConvertApp] Writing 86088 bytes to notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', 'notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', 'notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 83254 bytes to Pandora_Papers.pdf
